# 부도기업예측을 위한 재무변수 전처리

1. 거래소번호 수 6자리 맞추기
2. nan값 해결

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

In [24]:
train = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\재무데이터 최종.csv' , index_col = 0 )

## KIND에서 추출한 상폐관련 공시 리스트 CSV화

In [13]:
d0002 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0002.csv')
d0305 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0305.csv')
d0608 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0608.csv')
d0911 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0911.csv')
d1214 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1214.csv')
d1517 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1517.csv')
d1820 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1820.csv')

In [15]:
bankrupt_df = pd.concat([d0002, d0305,d0608, d0911, d1214, d1517, d1820], axis=0).reset_index()
bankrupt_df.drop(columns=['index'], inplace=True)

In [25]:
bankrupt_df

,시간,회사명,종목코드,공시제목,제출인
0,2002-10-31 18:27,정산애강,22220,부도발생및은행거래정지,정산애강
1,2002-10-08 12:06,대호특수강,21040,은행거래재개,대호특수강
2,2002-05-21 18:16,에이치엘비,28300,회사정리절차종결결정,에이치엘비
3,2002-04-17 17:42,에이치엘비,28300,회사정리절차종결신청,에이치엘비
4,2001-12-26 14:58,대호특수강,21040,회사정리절차개시결정,대호특수강
...,...,...,...,...,...
389,2020-02-12 16:21,에스앤더블류,103230,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에스앤더블류
390,2020-02-11 17:02,파나진,46210,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,파나진
391,2020-02-10 16:32,국순당,43650,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정],국순당
392,2020-02-10 16:24,유아이디,69330,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,유아이디


In [27]:
#저장
bankrupt_df.to_csv('kind_bankrupt_df.csv')

In [45]:
train['거래소코드'].dtype

dtype('int64')

In [54]:
print(type(str(train['거래소코드'])))

<class 'str'>


In [56]:
train['거래소코드']

0        58820
1        58820
2        58820
3        58820
4        58820
         ...  
27054    24060
27055    24060
27056    24060
27057    24060
27058    24060
Name: 거래소코드, Length: 27059, dtype: int64

In [55]:
str(train['거래소코드'])

'0        58820\n1        58820\n2        58820\n3        58820\n4        58820\n         ...  \n27054    24060\n27055    24060\n27056    24060\n27057    24060\n27058    24060\nName: 거래소코드, Length: 27059, dtype: int64'

In [52]:
train['거래소코드'].zfill(6)

AttributeError: 'Series' object has no attribute 'zfill'

In [68]:
!pip install openpyxl

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [71]:
import xlrd
budo = xlrd.open_workbook(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유00~02.xls')

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\r\n\r\n\r\n\r\n'

In [64]:
budo = pd.read_excel(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유00~02.xls')

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [21]:
train

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,...,이연부채(요약),자본총계(요약),자본금(요약),자본잉여금(요약),자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약),*미처분이익잉여금 및 차기이월미처분이익잉여금(요약),*당기순이익(요약),부채와자본총계(요약)
0,(주)CMG제약,58820,2000/12,5.0,NaN,UQ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820,2001/12,5.0,NaN,UQ,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820,2002/12,5.0,NaN,UQ,-26.34,-17.72,-39.61,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820,2003/12,5.0,NaN,UQ,-28.17,-79.37,-13.62,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820,2004/12,5.0,NaN,UQ,-53.18,-25.94,-54.38,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27054,흥구석유(주),24060,2004/12,5.0,NaN,UQ,0.21,0.07,-2.43,26.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27055,흥구석유(주),24060,2006/12,5.0,NaN,UQ,1.60,1.69,24.04,-22.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27056,흥구석유(주),24060,2002/12,5.0,NaN,UQ,4.71,-1.19,49.44,2.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27057,흥구석유(주),24060,2001/12,5.0,NaN,UQ,-8.26,-0.72,-33.74,2973.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
train['거래소코드'].value_counts().sort_index()

250       21
380        4
440       21
1000      21
1540      21
          ..
950170     4
950180     3
950190     3
950200     2
950220     2
Name: 거래소코드, Length: 2058, dtype: int64

In [15]:
train[train['회사명']== '(주)CMG제약']

,회사명,거래소코드,회계년도,유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),단기투자자산(요약),매출채권(요약),기타당좌자산(요약),비유동자산(요약),...,비유동부채(요약),사채(요약),장기차입금(요약),기타비유동부채(요약),자본총계(요약),자본금(요약),자본잉여금(요약),자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약)
0,(주)CMG제약,58820,2000/12,21169.0,10541.0,4330.0,4552.0,786.0,873.0,7821.0,...,0.0,5482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820,2001/12,16359.0,2307.0,1829.0,18.0,14.0,446.0,4549.0,...,0.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820,2002/12,9880.0,1436.0,355.0,246.0,26.0,809.0,5522.0,...,0.0,-2144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820,2003/12,8534.0,1083.0,2.0,381.0,72.0,628.0,2529.0,...,0.0,-2246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820,2004/12,3893.0,445.0,11.0,0.0,33.0,400.0,1286.0,...,0.0,-6119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,(주)CMG제약,58820,2005/12,3449.0,3226.0,609.0,1500.0,264.0,853.0,4831.0,...,0.0,-8677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,(주)CMG제약,58820,2006/12,6897.0,6897.0,2850.0,0.0,2400.0,1647.0,3133.0,...,0.0,-12088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,(주)CMG제약,58820,2007/12,8952.0,8638.0,2714.0,3234.0,2122.0,568.0,13122.0,...,0.0,-12978.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,(주)CMG제약,58820,2008/12,16725.0,15015.0,6233.0,926.0,7267.0,590.0,17832.0,...,0.0,-18256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,(주)CMG제약,58820,2009/12,9827.0,7398.0,1063.0,1105.0,4351.0,879.0,13008.0,...,0.0,-37358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for